In [1]:
!pip install nvcc4jupyter

In [2]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpx0mtm9rw".


In [3]:
%%writefile cuda_kernel.cu
#include <cuda_runtime.h>
#include <cmath>
#include <iostream>
#include <vector>
#include <iomanip>
#include <cstdlib>

#define WARP_SIZE 32

// math functions according to the problem statement
__device__ __forceinline__ float fast_sin(float x) { return __sinf(x); }
__device__ __forceinline__ float fast_cos(float x) { return __cosf(x); }
__device__ __forceinline__ float fast_log(float x) { return __logf(x); }
__device__ __forceinline__ float fast_exp(float x) { return __expf(x); }

__global__ void transform_kernel(const float4* __restrict__ input, float4* __restrict__ output, float* __restrict__ global_sum, int n) {

    // shared memory for both data and results
    extern __shared__ float shared_data[];

    const int tid = threadIdx.x;
    const int lane = tid % WARP_SIZE;
    const int warp_id = tid / WARP_SIZE;
    const int block_size = blockDim.x;
    const int block_start = blockIdx.x * block_size * 4;
    const int warp_start = block_start + warp_id * 32;

    // each thread loading one element for its warp
    for (int i = lane; i < 32; i += WARP_SIZE) {
        const int global_idx = warp_start + i;
        if (global_idx < n) {
            const int vec_idx = global_idx / 4;
            const int elem_idx = global_idx % 4;
            float4 data = input[vec_idx];

            // extracting correct component from the float4
            switch (elem_idx) {
                case 0: shared_data[warp_id * 32 + i] = data.x; break;
                case 1: shared_data[warp_id * 32 + i] = data.y; break;
                case 2: shared_data[warp_id * 32 + i] = data.z; break;
                case 3: shared_data[warp_id * 32 + i] = data.w; break;
            }
        }
    }
    __syncthreads();

    float warp_sin_sum = 0.0f;
    const int local_idx = warp_id * 32 + lane;
    const int global_idx = warp_start + lane;

    if (global_idx < n) {
        const int pos_in_32 = lane;
        const int mod_4 = pos_in_32 % 4;
        const float current_val = shared_data[local_idx];
        float result = 0.0f;

        // applying transformations based on position in 32 element block
        if (pos_in_32 < 16) {
            switch (mod_4) {
                case 0: result = fast_sin(current_val); break;
                case 1: result = fast_cos(current_val); break;
                case 2: result = (current_val > 0.0f) ? fast_log(current_val) : -INFINITY; break;
                case 3: result = fast_exp(current_val); break;
            }
        } else {
            const int ref_pos = pos_in_32 - 16;
            const float ref_val = shared_data[warp_id * 32 + ref_pos];
            float ref_result = 0.0f;

            // computing reference transformation
            switch (ref_pos % 4) {
                case 0: ref_result = fast_sin(ref_val); break;
                case 1: ref_result = fast_cos(ref_val); break;
                case 2: ref_result = (ref_val > 0.0f) ? fast_log(ref_val) : -INFINITY; break;
                case 3: ref_result = fast_exp(ref_val); break;
            }

            // computing current transformation and multiplying with reference
            switch (mod_4) {
                case 0: result = ref_result * fast_sin(current_val); break;
                case 1: result = ref_result * fast_cos(current_val); break;
                case 2: result = (current_val > 0.0f) ? (ref_result * fast_log(current_val)) : -INFINITY; break;
                case 3: result = ref_result * fast_exp(current_val); break;
            }
        }
        shared_data[local_idx + block_size * 4] = result;

        // gloabl sum calculation of sin terms according to the problem
        if (mod_4 == 0 && lane < 31) {
            const int cos_idx = local_idx + 1;
            float cos_val = shared_data[cos_idx + block_size * 4];
            if (cos_val > 0.5f) {
                warp_sin_sum += result;
            }
        }
    }
    __syncthreads();

    // back to global memory
    for (int i = lane; i < 32; i += WARP_SIZE) {
        const int global_idx = warp_start + i;
        if (global_idx < n) {
            const int vec_idx = global_idx / 4;
            const int elem_idx = global_idx % 4;
            float4* output_vec = &output[vec_idx];
            float result_val = shared_data[warp_id * 32 + i + block_size * 4];
            switch (elem_idx) {
                case 0: output_vec->x = result_val; break;
                case 1: output_vec->y = result_val; break;
                case 2: output_vec->z = result_val; break;
                case 3: output_vec->w = result_val; break;
            }
        }
    }
    for (int offset = 16; offset > 0; offset >>= 1) {
        warp_sin_sum += __shfl_down_sync(0xFFFFFFFF, warp_sin_sum, offset);
    }
    if (lane == 0 && warp_sin_sum != 0.0f) {
        atomicAdd(global_sum, warp_sin_sum);
    }
}

void measure_performance(int n, int num_runs = 4) {
    float4 *d_input, *d_output;
    float *d_global_sum, *d_temp;
    int n_float4 = (n + 3) / 4;
    size_t data_size_bytes = n * sizeof(float);
    double data_transferred_bytes = 2.0 * n * sizeof(float); // Read + Write
    std::cout << "performance measurement\n";
    std::cout << "array size: " << n << "("
              << std::fixed << std::setprecision(2)
              << (data_size_bytes / (1024.0 * 1024.0)) << " MB)\n";

    // memory allocation
    cudaMalloc(&d_input, n_float4 * sizeof(float4));
    cudaMalloc(&d_output, n_float4 * sizeof(float4));
    cudaMalloc(&d_global_sum, sizeof(float));
    cudaMalloc(&d_temp, data_size_bytes);

    // input data generation
    std::vector<float4> h_input_float4(n_float4);
    for (int i = 0; i < n_float4; i++) {
        float4 element;
        element.x = 0.1f + static_cast<float>(rand()) / (static_cast<float>(RAND_MAX/4.9f));
        element.y = 0.1f + static_cast<float>(rand()) / (static_cast<float>(RAND_MAX/4.9f));
        element.z = 0.1f + static_cast<float>(rand()) / (static_cast<float>(RAND_MAX/4.9f));
        element.w = 0.1f + static_cast<float>(rand()) / (static_cast<float>(RAND_MAX/4.9f));
        h_input_float4[i] = element;
    }
    cudaMemcpy(d_input, h_input_float4.data(), n_float4 * sizeof(float4), cudaMemcpyHostToDevice);

    // using settings from previous iterations (block_size)
    const int block_size = 256;
    const int grid_size = (n_float4 + block_size - 1) / block_size;
    const int shared_mem_size = 2 * block_size * 4 * sizeof(float);

    std::cout << "block size: " << block_size << " threads\n";
    std::cout << "shared memory: " << (shared_mem_size / 1024.0) << " KB per block\n";

    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    float total_time = 0.0f;
    float global_sum_result = 0.0f;

    std::cout << "\nrunning kernel " << num_runs << " time\n";

    for (int run = 0; run < num_runs; run++) {
        float h_global_sum = 0.0f;
        cudaMemcpy(d_global_sum, &h_global_sum, sizeof(float), cudaMemcpyHostToDevice);

        cudaEventRecord(start);
        transform_kernel<<<grid_size, block_size, shared_mem_size>>>(d_input, d_output, d_global_sum, n);
        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        cudaError_t err = cudaGetLastError();
        if (err != cudaSuccess) {
            std::cout << "kernel error: " << cudaGetErrorString(err) << std::endl;
            continue;
        }

        float kernel_time_ms = 0.0f;
        cudaEventElapsedTime(&kernel_time_ms, start, stop);
        total_time += kernel_time_ms;

        if (run == 0) {
            cudaMemcpy(&h_global_sum, d_global_sum, sizeof(float), cudaMemcpyDeviceToHost);
            global_sum_result = h_global_sum;
        }
    }

    float avg_time = total_time / num_runs;
    double kernel_bandwidth = (data_transferred_bytes / (avg_time / 1000.0)) / (1024.0 * 1024.0 * 1024.0);

    cudaEventRecord(start);
    cudaMemcpy(d_temp, d_input, data_size_bytes, cudaMemcpyDeviceToDevice);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float memcpy_time_ms = 0.0f;
    cudaEventElapsedTime(&memcpy_time_ms, start, stop);
    double memcpy_bandwidth = (data_transferred_bytes / (memcpy_time_ms / 1000.0)) / (1024.0 * 1024.0 * 1024.0);
    double efficiency = (kernel_bandwidth / memcpy_bandwidth) * 100.0;

    // results
    std::cout << "\nperformance results:\n";
    std::cout << "kernel execution time: " << std::fixed << std::setprecision(3) << avg_time << " ms\n";
    std::cout << "kernel bandwidth: " << std::fixed << std::setprecision(2) << kernel_bandwidth << " GB/s\n";
    std::cout << "memory bandwidth: " << std::fixed << std::setprecision(2) << memcpy_bandwidth << " GB/s\n";
    std::cout << "efficiency: " << std::fixed << std::setprecision(1) << efficiency << "%\n";
    std::cout << "global sin sum: " << std::scientific << std::setprecision(6) << global_sum_result << std::endl;

    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_global_sum);
    cudaFree(d_temp);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
}

int main() {
    const int n = 100000000;
    const int num_runs = 4;
    srand(42);

    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    std::cout << "using google coab for gpu: " << prop.name << "\n";
    std::cout << "global memory: " << (prop.totalGlobalMem / (1024.0 * 1024.0 * 1024.0)) << " GB\n";
    std::cout << "shared memory per block: " << (prop.sharedMemPerBlock / 1024.0) << " KB\n";
    std::cout << std::endl;

    measure_performance(n, num_runs);
    return 0;
}

Writing cuda_kernel.cu


In [4]:
!nvcc -o cuda_kernel cuda_kernel.cu -std=c++17 -arch=sm_80 -O3 --use_fast_math -Xptxas -O3

In [5]:
!./cuda_kernel

using google coab for gpu: NVIDIA A100-SXM4-40GB
global memory: 39.5574 GB
shared memory per block: 48 KB

performance measurement
array size: 100000000(381.47 MB)
 block size: 256 threads
 shared memory: 8.00 KB per block

running kernel 4 time

performance results:
kernel execution time: 1.958 ms
kernel bandwidth: 380.54 GB/s
memory bandwidth: 492.62 GB/s
efficiency: 77.2%
global sin sum: 1.012631e+05
